In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


# from tflite_model_maker import model_spec
# from tflite_model_maker import image_classifier
# from tflite_model_maker.config import ExportFormat
# from tflite_model_maker.config import QuantizationConfig
# from tflite_model_maker.image_classifier import DataLoader
# import sys
# sys.modules[__name__].__dict__.clear()


In [2]:
# data=DataLoader.from_folder("/nfs/general/shared/")
# train_data, rest_data = data.split(0.8)


aside=np.array([mpimg.imread(x) for x in glob.glob('/nfs/general/shared/KeyA/*.jpg')])
bside=np.array([mpimg.imread(x) for x in glob.glob('/nfs/general/shared/KeyD/*.jpg')])

def ExtractRoi(img):   
    img=tf.cast(img[220:320, 260:400, :], tf.float32)-150
    return tf.cast(tf.maximum(img, tf.zeros_like(img))+150, tf.uint8)

print(aside.size)
print(bside.size)

# ExtractRoi(aside[0,:,:,:])

235008000
1607270400


In [4]:
from sklearn.cluster import KMeans

# X = np.concatenate((
#     aside.reshape((-1, 3)),
#     bside.reshape((-1, 3))))

X=tf.vectorized_map(ExtractRoi, tf.concat((aside,bside), axis=0))
X=tf.reshape(X, (-1,3)).numpy()
kmeans = KMeans(n_clusters=32, random_state=0, verbose=1, max_iter=300, n_init=10).fit(X)


Initialization complete
Iteration 0, inertia 3087991305.0
Iteration 1, inertia 2675156221.6681423
Iteration 2, inertia 2597291189.2385855
Iteration 3, inertia 2563229781.509137
Iteration 4, inertia 2545715512.174677
Iteration 5, inertia 2534282227.1459193
Iteration 6, inertia 2525134857.46408
Iteration 7, inertia 2516258882.1179457
Iteration 8, inertia 2507372928.6404943
Iteration 9, inertia 2498411950.6140895
Iteration 10, inertia 2490436848.1372614
Iteration 11, inertia 2483211672.87029
Iteration 12, inertia 2475731464.42902
Iteration 13, inertia 2468927278.40789
Iteration 14, inertia 2463358738.5431504
Iteration 15, inertia 2459082615.2343545
Iteration 16, inertia 2455671090.3151345
Iteration 17, inertia 2453176452.0142756
Iteration 18, inertia 2451495278.475019
Iteration 19, inertia 2450122793.7749662
Iteration 20, inertia 2448998001.4186106
Iteration 21, inertia 2448184457.613324
Iteration 22, inertia 2447486114.53958
Iteration 23, inertia 2446821697.514733
Iteration 24, inertia 2

In [5]:
centroids = tf.constant(kmeans.cluster_centers_, dtype=tf.float32)

class Quantizer(tf.Module):
    def QuantizeImagePixels(self, sample_image):
        sample_image = ExtractRoi(sample_image)
        sample_image_as_vec = tf.cast(
            tf.reshape(sample_image, (-1, 3)), tf.float32)
        cov_mat = -2.0*tf.matmul(sample_image_as_vec,
                                 centroids, transpose_a=False, transpose_b=True)
        labels = tf.argmin(cov_mat + tf.reduce_sum(sample_image_as_vec*sample_image_as_vec,
                           1, keepdims=True) + tf.reduce_sum(centroids*centroids, 1), axis=1)
        return tf.reshape(labels, sample_image.shape[:2])

    def QuantizeHist(self, sample_image):
        sample_image = ExtractRoi(sample_image)
        sample_image_as_vec = tf.cast(
            tf.reshape(sample_image, (-1, 3)), tf.float32)
        cov_mat = -2.0*tf.matmul(sample_image_as_vec,
                                 centroids, transpose_a=False, transpose_b=True)
        labels = tf.argmin(cov_mat + tf.reduce_sum(sample_image_as_vec*sample_image_as_vec,
                           1, keepdims=True) + tf.reduce_sum(centroids*centroids, 1), axis=1)
        return tf.reduce_sum(tf.one_hot(labels, centroids.shape[0]), axis=0)


# 

# for i in [2,3,4]:
#     plt.imshow(ExtractRoi(aside[i, :, :, :]))
#     plt.show()
#     plt.imshow(ExtractRoi(bside[i, :, :, :]))
#     plt.show()


In [10]:
# from sklearn.decomposition import PCA
# foo=PCA(n_components=5).fit_transform(tf.concat((a_quantized,b_quantized),axis=0).numpy())
# plt.scatter(foo[:,0],foo[:,1],color='r')

# plt.scatter(foo[:a_quantized.shape[0],0],foo[:a_quantized.shape[0],1],color='g')

m = Quantizer()
a_quantized=tf.vectorized_map(m.QuantizeHist, aside)
b_quantized=tf.vectorized_map(m.QuantizeHist, bside)


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

X=tf.concat((a_quantized,b_quantized),axis=0)

y=tf.concat((tf.zeros((a_quantized.shape[0],)),tf.ones((b_quantized.shape[0],))),axis=0)
model.fit(X, y, epochs=5000)
# fashion_mnist = tf.keras.datasets.fashion_mnist

# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



# 


Epoch 1/5000
63/63 [==============================] - 0s 533us/step - loss: 46.1805 - accuracy: 0.8629
Epoch 2/5000
63/63 [==============================] - 0s 637us/step - loss: 0.7853 - accuracy: 0.9955
Epoch 3/5000
63/63 [==============================] - 0s 668us/step - loss: 0.7259 - accuracy: 0.9965
Epoch 4/5000
63/63 [==============================] - 0s 709us/step - loss: 0.6237 - accuracy: 0.9960
Epoch 5/5000
63/63 [==============================] - 0s 725us/step - loss: 0.6276 - accuracy: 0.9945
Epoch 6/5000
63/63 [==============================] - 0s 641us/step - loss: 0.5707 - accuracy: 0.9955
Epoch 7/5000
63/63 [==============================] - 0s 644us/step - loss: 0.4273 - accuracy: 0.9970
Epoch 8/5000
63/63 [==============================] - 0s 714us/step - loss: 0.5585 - accuracy: 0.9945
Epoch 9/5000
63/63 [==============================] - 0s 663us/step - loss: 0.3866 - accuracy: 0.9980
Epoch 10/5000
63/63 [==============================] - 0s 619us/step - loss: 0.37

In [7]:
class ImageClassifier(tf.Module):
    def __init__(self):
      self.classifier = probability_model
      self.quantizer = Quantizer()
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[480, 640, 3], dtype=tf.uint8)])
    def Classify(self, sample_image):
      #return probability_model(tf.expand_dims(self.quantizer.QuantizeHist(sample_image), 0))
      return self.classifier(tf.expand_dims(self.quantizer.QuantizeHist(sample_image),0))


model = ImageClassifier()

tf.saved_model.save(model, '/tmp/classifier')
converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/classifier') # path to the SavedModel directory
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#   tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open('/nfs/general/shared/adder.tflite', 'wb').write(tflite_model)


# model_spec.IMAGE_CLASSIFICATION_MODELS
# spec=model_spec.get('efficientnet_lite4')

# spec.input_image_shape

2022-04-18 06:17:21.645663: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/classifier/assets
Estimated count of arithmetic ops: 3.712 M  ops, equivalently 1.856 M  MACs


2022-04-18 06:17:22.271461: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-18 06:17:22.271481: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-18 06:17:22.272020: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/classifier
2022-04-18 06:17:22.273158: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-18 06:17:22.273166: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/classifier
2022-04-18 06:17:22.276387: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-18 06:17:22.300056: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/classifier
2022-04-18 06:17:22.314528: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 42509 micr

9336

In [8]:
model.Classify(aside[10,:,:,:])

# validation_data, test_data = rest_data.split(0.5)
# model = image_classifier.create(train_data, model_spec=model_spec.get('efficientnet_lite4'), validation_data=validation_data, epochs = 20)


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 0.]], dtype=float32)>

In [7]:
class Adder(tf.Module):
  @tf.function(input_signature=[tf.TensorSpec(shape=[480,640], dtype=tf.uint8)])
  def add(self, image):
    image = image[tf.newaxis, ..., tf.newaxis]
    return image

model = Adder()
tf.saved_model.save(model, '/tmp/adder')


converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/adder') # path to the SavedModel directory
tflite_model = converter.convert()

with open('/nfs/general/shared/adder.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/adder/assets


INFO:tensorflow:Assets written to: /tmp/adder/assets
2022-04-16 15:17:41.245018: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-16 15:17:41.245037: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs


2022-04-16 15:17:41.245221: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/adder
2022-04-16 15:17:41.245395: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-16 15:17:41.245403: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/adder
2022-04-16 15:17:41.245820: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-16 15:17:41.254473: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/adder
2022-04-16 15:17:41.257203: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 11982 microseconds.
2022-04-16 15:17:41.268568: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs

